# References


- https://api.kivaws.org/
- https://groups.google.com/g/build-kiva/c/rh3XstWG4q0/m/57kXwvmgAAAJ?pli=1
- http://www.kivalens.org/graphql?query=%23compound%20query%20with%20query%20variables.%0Aquery%20compoundQuery(%24ids%3A%20%5BInt%5D!)%20%7B%0A%20%20%0A%20%20%23aliased%20results%0A%20%20luisEducation%3A%20loan(id%3A1063113)%20%7B%0A%20%20%20%20...basicLoan%0A%20%20%7D%0A%20%20%0A%20%20%23using%20the%20query%20variables%20and%20fragments%0A%20%20loans(ids%3A%20%24ids)%20%7B%0A%20%20%20%20...basicLoan%0A%20%20%20%20%0A%20%20%20%20%23I%20added%20support%20for%20it%20to%20reference%20partner%20objects%0A%20%20%20%20partner%20%7B%0A%20%20%20%20%20%20...basicPartner%0A%20%20%20%20%20%20start_date%0A%20%20%20%20%7D%0A%20%20%20%20%0A%20%20%20%20%23this%20makes%20an%20API%20call%20to%20Kiva%2C%20similar%20loans%20are%20not%20cached%20on%20KL%20server%0A%20%20%20%20similar%20%7B%0A%20%20%20%20%20%20...basicLoan%0A%20%20%20%20%20%20location%20%7B%0A%20%20%20%20%20%20%20%20country%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%20%20%0A%20%20%20%20%20%20%23this%20gets%20the%20similiar%20loans%20of%20similiar%20loans%0A%20%20%20%20%20%20similar%20%7B%0A%20%20%20%20%20%20%20%20...basicLoan%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%20%20%0A%20%20%23find%20all%20partners%20in%20north%20america%0A%20%20partners(criteria%3A%20%7Bregion%3A%20%22na%22%7D)%20%7B%0A%20%20%20%20...basicPartner%0A%20%20%7D%0A%7D%0A%0A%23fragments%20to%20avoid%20repetition%20above%0Afragment%20basicPartner%20on%20Partner%20%7B%0A%20%20id%0A%20%20name%0A%7D%0A%0Afragment%20basicLoan%20on%20Loan%20%7B%0A%20%20id%0A%20%20name%0A%20%20sector%0A%7D&operationName=compoundQuery&variables=%7B%22ids%22%3A%20%5B1063117%2C%201067053%5D%7D
- https://www.kiva.org/build/data-snapshots
- https://api.kivaws.org/graphql?query=%7B%0A%20%20lend%20%7B%0A%20%20%20%20loans(filters%3A%20%7Bstatus%3A%20fundraising%7D%2C%20sortBy%3A%20newest)%20%7B%0A%20%20%20%20%20%20values%20%7B%0A%20%20%20%20%20%20%20%20id%0A%20%20%20%20%20%20%20%20name%0A%20%20%20%20%20%20%20%20loanAmount%0A%20%20%20%20%20%20%7D%0A%20%20%20%20%7D%0A%20%20%7D%0A%7D%0A&operationName=null
- https://groups.google.com/g/build-kiva
- https://gist.github.com/lovingawareness/3161250
- https://docs-eas.leanix.net/docs/graphql-in-10-minutes
- https://marketplace-api.k1.kiva.org/graphql


# I. Setup

Before running the code below, please specify the following parameters:

#### 1. Number of batches of data
Each batch of data contains information about a 100 loans and all lenders, borrowers. teams, etc. related to the loans

In [ ]:
n_batches = 300

#### 2. Filtering conditions
The queries sent to Kiva`s Api can be filtered in accordance with user requirements. There are several parameters that have to be specified:

1. `offset`: Int = 0. Represents how many loans should the API skip
2. `limit`: Int = 20.  Represents how many loans should the API select. Max value = 100
3. `filters`: LoanSearchFiltersInput = { "distributionModel": "both", "status": "fundraising" } Represents filtering conditions. Contains:
    - `activity`: Int. Single activityId or array of them
    - `arrearsRate`: {min:0 max: 100}. A two-value array of {min, max} values (inclusive) for arrearsRate. A typical range is 0 - 0.5.
    - `avgBorrowerCost`: {min:0 max: 100}. A two-value array of {min, max} values (inclusive) for avgBorrowerCost. A typical range is 0 - 95.
    - `cityState`: String. City and state combo to search, e.g. San Francisco,CA
    - `country`: String. Single country ISO-code or array of them
    - `currencyLossPossible`: Boolean. If the loan has Currency exchange loss.
    - `dafEligible`: Boolean. If the loan is marked as daf_eligible for the DAF program (true / false)
    - `defaultRate`: {min:0 max: 100}. A two-value array of {min, max} values (inclusive) for defaultRate. A typical range is 0 - 0.2.
    - `distributionModel`: DistributionModelEnum = "both". If the loan is direct, fieldPartner or both
    - `expiringSoon`: Boolean. Whether we should filter for loans nearing expiry as defined by Kiva\Common\Conf value for expiring_soon_window (true / false)
    - `gender`: GenderEnum. 'male' or 'female'
    - `hasResearchScore`: Boolean. If the research score for the loan theme instance of the loan is greater than or equal to 10
    - `isGroup`: Boolean. group loan or individual loan?
    - `lenderFavorite`: Int. The Kiva lender ID to search for to see if the loan is a favorite of theirs
    - `lenderTerm`: {min:0 max: 100}. A two-value array of {min, max} integer values (inclusive) for lenderRepaymentTerm. A typical range is 6 - 24.
    - `loanIds`: Int. List of Kiva loan IDs. Not savable to Loan Search Criteria. Specify a status in the search filters to use this field for loans not in fundRaising
    - `loanLimit`: Int. The maximum amount that may be lent to a loan
    - `loanTags`: Int. The tag IDs for tags that have been successfully applied to the loan.
    - `matcherAccountId`: Int. Loan matcher's account ID, or an array of them
    - `partner`: Int. Single partnerID or array of them, used to specify partners that should be INCLUDED from the search
    - `profitability`: {min:0 max: 100}. A two-value array of {min, max} values (inclusive) for avgProfitability. A typical range is -50 - 50
    - `riskRating`: {min:0 max: 100}. Array of risk rating values from 0.0 to 5.0. e.g. [0.5,2.5]
    - `sector`: Int. Single sector ID or array of them.
    - `status`: LoanSearchStatusEnum = "fundraising". Loan status to search for - fundraising, funded, all.
    - `theme`: String. Single theme or array of them to be OR'd, e.g. 'water' or array('green', 'water')
    - `trustee`: Int. Single trustee ID or an array of them
    - `isMatched`: Boolean. Whether to select only matched loans or not

4. sortBy: LoanSearchSortByEnum = "popularity" = popularity
Represents how loans are selected. Accepts values `amountLeft`, `expiringSoon`, `loanAmount`, `loanAmountDesc`, `newest`, `popularity`, `random`, `repaymentTerm`.


|Name|Type|Description|Example|
|:---|:---|:-----|:---|
|activity| Int|Single activityId or array of them||
|arrearsRate|{min:0 max: 100}|A two-value array of {min, max} values (inclusive) for arrearsRate. A typical range is 0-0.5.||
|avgBorrowerCost|{min:0 max: 100}|A two-value array of {min, max} values (inclusive) for avgBorrowerCost. A typical range is 0-95.||
|cityState| String|City and state combo to search, e.g. San Francisco,CA||
|country| String|Single country ISO-code or array of them||
|currencyLossPossible| Boolean|If the loan has Currency exchange loss.||
|dafEligible| Boolean|If the loan is marked as daf_eligible for the DAF program (true / false)||
|defaultRate| {min:0 max: 100}|A two-value array of {min, max} values (inclusive) for defaultRate. A typical range is 0-0.2.||
|distributionModel| DistributionModelEnum = "both"|If the loan is direct, fieldPartner or both||
|expiringSoon| Boolean|Whether we should filter for loans nearing expiry as defined by Kiva\Common\Conf value for expiring_soon_window (true / false)||
|gender| GenderEnum|male' or 'female'||
|hasResearchScore| Boolean|If the research score for the loan theme instance of the loan is greater than or equal to 10||
|isGroup| Boolean|group loan or individual loan?||
|lenderFavorite| Int|The Kiva lender ID to search for to see if the loan is a favorite of theirs||
|lenderTerm|{min:0 max: 100}|A two-value array of {min, max} integer values (inclusive) for lenderRepaymentTerm. A typical range is 6-24.||
|loanIds| Int|List of Kiva loan IDs. Not savable to Loan Search Criteria. Specify a status in the search filters to use this field for loans not in fundRaising||
|loanLimit| Int|The maximum amount that may be lent to a loan||
|loanTags| Int|The tag IDs for tags that have been successfully applied to the loan.||
|matcherAccountId| Int|Loan matcher's account ID, or an array of them||
|partner| Int|Single partnerID or array of them, used to specify partners that should be INCLUDED from the search||
|profitability|{min:0 max: 100}|A two-value array of {min, max} values (inclusive) for avgProfitability. A typical range is -50-50||
|riskRating|{min:0 max: 100}|Array of risk rating values from 0.0 to 5.0. e.g. [0.5,2.5]||
|sector| Int|Single sector ID or array of them.||
|status| LoanSearchStatusEnum = "fundraising"|Loan status to search for fundraising, funded, all.||
|theme| String|Single theme or array of them to be OR'd, e.g. 'water' or array('green', 'water')||
|trustee| Int|Single trustee ID or an array of them||
|isMatched| Boolean|Whether to select only matched loans or not||

In [ ]:
filter_query = '''
                    offset: 0
                    limit: 1000
                    filters: {distributionModel: both, 
                              status: all}
                    sortBy: random
                '''

### 1. Libraries

In [ ]:
import requests
import pandas as pd
import time

### 2. Create empty dictionaries for storing information

In [ ]:
def create_empty_dict():

    #======================================================
    # Loans dict
    loan_dict= {
            # LoanBasic object
                'loan_anonymizationLevel':[],
                'loan_borrowerCount':[],
                'loan_dafEligible':[],
                'loan_delinquent':[],
                'loan_description':[],
                'loan_descriptionInOriginalLanguage':[],
                'loan_disbursalDate':[],
                'loan_distributionModel':[],
                'loan_fundraisingDate':[],
                'loan_gender':[],
                'loan_hasCurrencyExchangeLossLenders':[],
                'loan_id':[],                              # primary key
                'loan_inPfp':[],
                'loan_isMatchable':[],
                'loan_lenderRepaymentTerm':[],
                'loan_loanAmount':[],
                'loan_matcherAccountId':[],
                'loan_matcherName':[],
                'loan_matchingText':[],
                'loan_matchRatio':[],
                'loan_minNoteSize':[],
                'loan_name':[],
                'loan_paidAmount':[],
                'loan_pfpMinLenders':[],
                'loan_plannedExpirationDate':[],
                'loan_previousLoanId':[],
                'loan_raisedDate':[],
                'loan_repaymentInterval':[],
                'loan_researchScore':[],
                'loan_status':[],
                'loan_tags':[],
                'loan_use':[],
                'loan_whySpecial':[],
                'loan_LenderNum':[],
                'loan_ActionNum':[],
            # LoanFundraisingInfo object
                'fundraising_fundedAmount':[],
                'fundraising_isExpiringSoon':[],
                'fundraising_reservedAmount':[],
            # Activity object
                'activity_id':[],                          # foreign key to Activity
                'activity_name':[],
            # Sector object
                'sector_id':[],                            # foreign key to Sector
                'sector_name':[],
            # Geocode object
                'geocode_latitude':[],                     # foreign key to Geocode
                'geocode_longitude':[]                     # foreign key to Geocode
                }

    #======================================================
    # Loan Term dict
    term_dict = {
            # LoanTerm object
                'term_currency':[],
                'term_currencyFullName':[],
                'term_disbursalAmount':[],
                'term_disbursalDate':[],
                'term_flexibleFundraisingEnabled':[],
                'term_lenderRepaymentTerm':[],
                'term_loanAmount':[],
                'term_lossLiabilityCurrencyExchange':[],
                'term_lossLiabilityNonpayment':[],
            # LoanBasic object
                'loan_id':[]                               # foreign key to Loan
                }

    #======================================================
    # Loan Repayment dict 
    repayment_dict = {
            # RepaymentExpected object
                'repayment_amount':[],
                'repayment_dueToKivaDate':[],
                'repayment_effectiveDate':[],
                'repayment_localAmount':[],
            # LoanBasic object
                'loan_id':[]                               # foreign key to Loan
                }

    #======================================================
    # Borrowers dict
    borrower_dict = {
            # Borrower object
                'borrower_borrowedAmount':[],
                'borrower_firstName':[],
                'borrower_gender':[],
                'borrower_id':[],                          # primary key
                'borrower_isPrimary':[],
                'borrower_pictured':[]
                }

    # Loan-Borrower relationship dict
    LoanBorrowerRelationship_dict = {'loan_id':[],
                                     'borrower_id':[]}

    #======================================================
    # Lenders dict
    lender_dict = {
            # Lender object
                'lender_id':[],                            # primary key
                'lender_inviteeCount':[],
                'lender_loanCount':[],
                'lender_memberSince':[],
                'lender_name':[],
                'lender_publicId':[],
            # LenderPage object
                'page_city':[],
                'page_loanBecause':[],
                'page_occupation':[],
                'page_otherInfo':[],
                'page_state':[],
                'page_url':[],
                'page_whereabouts':[],
            # Country object
                'country_isoCode':[]                       # foreign key to Country
                }

    # Loan-Lender relationship dict
    LoanLenderRelationship_dict = {'loan_id':[],
                                   'lender_id':[]}

    #======================================================
    # Teams dict
    team_dict = {
            # Team object
                'team_category':[],
                'team_createdDate':[],
                'team_description':[],
                'team_id':[],                              # primary key
                'team_lenderCount':[],
                'team_lentAmount':[],
                'team_loanBecause':[],
                'team_loanCount':[],
                'team_membershipType':[],
                'team_name':[],
                'team_teamPublicId':[],
                'team_url':[],
                'team_whereabouts':[]
                }

    # Lender-Team relationship dict
    LenderTeamRelationship_dict = {'lender_id':[],
                                   'team_id':[]}
    
    #======================================================
    # Lending Action dict
    action_dict = {
            # LendingAction object
                'action_latestSharePurchaseDate':[],
                'action_shareAmount':[],
                'action_teams':[],
            # Lender object
                'lender_id':[],                            # foreign key to Lender
                'loan_id':[]                               # foreign key to Loan
                }
    
    #======================================================
    # Geocode dict
    geocode_dict = {
            # Geocode object
                'geocode_city':[],
                'geocode_latitude':[],                     # primary key
                'geocode_longitude':[],                    # primary key
                'geocode_postalCode':[],
                'geocode_state':[],
            # Country object
                'country_isoCode':[]                       # foreign key to Country
                }

    #======================================================
    # Country dict
    country_dict = {
            # Country object
                'country_fundsLentInCountry':[],
                'country_isoCode':[],                      # primary key
                'country_name':[],
                'country_numLoansFundraising':[],
                'country_ppp':[],
                'country_region':[]
                }

    #======================================================
    # Activity dict
    activity_dict = {
            # Activity object
                'activity_id':[],                          # primary key
                'activity_name':[]
                }

    #======================================================
    # Sector dict
    sector_dict = {
            # Sector object
                'sector_id':[],                            # primary key
                'sector_name':[]
                }  
    
    return loan_dict, term_dict, repayment_dict, borrower_dict, LoanBorrowerRelationship_dict, lender_dict, LoanLenderRelationship_dict, action_dict, team_dict, LenderTeamRelationship_dict, geocode_dict, country_dict, activity_dict, sector_dict


### 3. Mapping API objects to dictionaries

In [ ]:
# Loans Table
def loan_obj_to_dict(in_obj, append_dict, n_lenders, n_lendingActions):
    # Check if observation is not in dict
    if in_obj['id'] not in append_dict['loan_id']:

        # LoanBasic object
        append_dict['loan_anonymizationLevel'].append(in_obj['anonymizationLevel'])
        append_dict['loan_borrowerCount'].append(in_obj['borrowerCount'])
        append_dict['loan_dafEligible'].append(in_obj['dafEligible'])
        append_dict['loan_delinquent'].append(in_obj['delinquent'])
        append_dict['loan_description'].append(in_obj['description'])
        append_dict['loan_descriptionInOriginalLanguage'].append(in_obj['descriptionInOriginalLanguage'])
        append_dict['loan_disbursalDate'].append(in_obj['disbursalDate'])
        append_dict['loan_distributionModel'].append(in_obj['distributionModel'])
        append_dict['loan_fundraisingDate'].append(in_obj['fundraisingDate'])
        append_dict['loan_gender'].append(in_obj['gender'])
        append_dict['loan_hasCurrencyExchangeLossLenders'].append(in_obj['hasCurrencyExchangeLossLenders'])
        append_dict['loan_id'].append(in_obj['id'])
        append_dict['loan_inPfp'].append(in_obj['inPfp'])
        append_dict['loan_isMatchable'].append(in_obj['isMatchable'])
        append_dict['loan_lenderRepaymentTerm'].append(in_obj['lenderRepaymentTerm'])
        append_dict['loan_loanAmount'].append(in_obj['loanAmount'])
        append_dict['loan_matcherAccountId'].append(in_obj['matcherAccountId'])
        append_dict['loan_matcherName'].append(in_obj['matcherName'])
        append_dict['loan_matchingText'].append(in_obj['matchingText'])
        append_dict['loan_matchRatio'].append(in_obj['matchRatio'])
        append_dict['loan_minNoteSize'].append(in_obj['minNoteSize'])
        append_dict['loan_name'].append(in_obj['name'])
        append_dict['loan_paidAmount'].append(in_obj['paidAmount'])
        append_dict['loan_pfpMinLenders'].append(in_obj['pfpMinLenders'])
        append_dict['loan_plannedExpirationDate'].append(in_obj['plannedExpirationDate'])
        append_dict['loan_previousLoanId'].append(in_obj['previousLoanId'])
        append_dict['loan_raisedDate'].append(in_obj['raisedDate'])
        append_dict['loan_repaymentInterval'].append(in_obj['repaymentInterval'])
        append_dict['loan_researchScore'].append(in_obj['researchScore'])
        append_dict['loan_status'].append(in_obj['status'])
        append_dict['loan_tags'].append(in_obj['tags'])
        append_dict['loan_use'].append(in_obj['use'])
        append_dict['loan_whySpecial'].append(in_obj['whySpecial'])
        append_dict['loan_LenderNum'].append(n_lenders)   
        append_dict['loan_ActionNum'].append(n_lendingActions)
        
        # loanFundraisingInfo object
        if in_obj['loanFundraisingInfo'] is not None:
            append_dict['fundraising_fundedAmount'].append(in_obj['loanFundraisingInfo']['fundedAmount'])
            append_dict['fundraising_isExpiringSoon'].append(in_obj['loanFundraisingInfo']['isExpiringSoon'])
            append_dict['fundraising_reservedAmount'].append(in_obj['loanFundraisingInfo']['reservedAmount'])
        else:
            append_dict['fundraising_fundedAmount'].append(None)
            append_dict['fundraising_isExpiringSoon'].append(None)
            append_dict['fundraising_reservedAmount'].append(None)

        # Activity object
        if in_obj['activity'] is not None:    
            append_dict['activity_id'].append(in_obj['activity']['id'])
            append_dict['activity_name'].append(in_obj['activity']['name'])
        else:
            append_dict['activity_id'].append(None)
            append_dict['activity_name'].append(None)

        # Sector object
        if in_obj['sector'] is not None:    
            append_dict['sector_id'].append(in_obj['sector']['id'])
            append_dict['sector_name'].append(in_obj['sector']['name'])
        else:
            append_dict['sector_id'].append(None)
            append_dict['sector_name'].append(None)

        # Geocode object
        if in_obj['geocode'] is not None:    
            append_dict['geocode_latitude'].append(in_obj['geocode']['latitude'])
            append_dict['geocode_longitude'].append(in_obj['geocode']['longitude'])
        else:
            append_dict['geocode_latitude'].append(None)
            append_dict['geocode_longitude'].append(None)
            
    return append_dict

#=============================================================================================================================
# Loan Terms Table
def term_obj_to_dict(in_obj, append_dict, loan_foreign_key):
    # Check if observation is not in dict
    if loan_foreign_key not in append_dict['loan_id']:  
        
        # LoanTerm object
        append_dict['term_currency'].append(in_obj['currency'])
        append_dict['term_currencyFullName'].append(in_obj['currencyFullName'])
        append_dict['term_disbursalAmount'].append(in_obj['disbursalAmount'])
        append_dict['term_disbursalDate'].append(in_obj['disbursalDate'])
        append_dict['term_flexibleFundraisingEnabled'].append(in_obj['flexibleFundraisingEnabled'])
        append_dict['term_lenderRepaymentTerm'].append(in_obj['lenderRepaymentTerm'])
        append_dict['term_loanAmount'].append(in_obj['loanAmount'])
        append_dict['term_lossLiabilityCurrencyExchange'].append(in_obj['lossLiabilityCurrencyExchange'])
        append_dict['term_lossLiabilityNonpayment'].append(in_obj['lossLiabilityNonpayment'])

        # Foreign key - LoanBasic
        append_dict['loan_id'].append(loan_foreign_key)

    return append_dict


#=============================================================================================================================
# Repayment Schedule Table
def repayment_obj_to_dict(in_obj, append_dict, loan_foreign_key):
    # Check if observation is not in dict
    if loan_foreign_key not in append_dict['loan_id']:  

        # RepaymentExpected object
        append_dict['repayment_amount'].append(in_obj['amount'])
        append_dict['repayment_dueToKivaDate'].append(in_obj['dueToKivaDate'])
        append_dict['repayment_effectiveDate'].append(in_obj['effectiveDate'])
        append_dict['repayment_localAmount'].append(in_obj['localAmount'])

        # Foreign key - LoanBasic
        append_dict['loan_id'].append(loan_foreign_key)

    return append_dict

#=============================================================================================================================
# Borrowers Table
def borrower_obj_to_dict(in_obj, append_dict):
    # Check if observation is not in dict
    if in_obj['id'] not in append_dict['borrower_id']:
        
        # Borrower object
        append_dict['borrower_borrowedAmount'].append(in_obj['borrowedAmount'])
        append_dict['borrower_firstName'].append(in_obj['firstName'])
        append_dict['borrower_gender'].append(in_obj['gender'])
        append_dict['borrower_id'].append(in_obj['id'])
        append_dict['borrower_isPrimary'].append(in_obj['isPrimary'])
        append_dict['borrower_pictured'].append(in_obj['pictured'])

    return append_dict

#=============================================================================================================================
# Lenders Table
def lender_obj_to_dict(in_obj, append_dict):
    # Check if observation is not in dict
    if in_obj['id'] not in append_dict['lender_id']:
        
        # Lender object
        append_dict['lender_id'].append(in_obj['id'])
        append_dict['lender_inviteeCount'].append(in_obj['inviteeCount'])
        append_dict['lender_loanCount'].append(in_obj['loanCount'])
        append_dict['lender_memberSince'].append(in_obj['memberSince'])
        append_dict['lender_name'].append(in_obj['name'])
        append_dict['lender_publicId'].append(in_obj['publicId'])

        # LenderPage object
        if in_obj['lenderPage'] is not None:
            append_dict['page_city'].append(in_obj['lenderPage']['city'])
            append_dict['page_loanBecause'].append(in_obj['lenderPage']['loanBecause'])
            append_dict['page_occupation'].append(in_obj['lenderPage']['occupation'])
            append_dict['page_otherInfo'].append(in_obj['lenderPage']['otherInfo'])
            append_dict['page_state'].append(in_obj['lenderPage']['state'])
            append_dict['page_url'].append(in_obj['lenderPage']['url'])
            append_dict['page_whereabouts'].append(in_obj['lenderPage']['whereabouts'])

            # Country object
            if in_obj['lenderPage']['country'] is not None:
                append_dict['country_isoCode'].append(in_obj['lenderPage']['country']['isoCode'])
            else:
                append_dict['country_isoCode'].append(None)

        else:
            append_dict['page_city'].append(None)
            append_dict['page_loanBecause'].append(None)
            append_dict['page_occupation'].append(None)
            append_dict['page_otherInfo'].append(None)
            append_dict['page_state'].append(None)
            append_dict['page_url'].append(None)
            append_dict['page_whereabouts'].append(None)

    return append_dict

#=============================================================================================================================
# Teams Table
def team_obj_to_dict(in_obj, append_dict):
    # Check if observation is not in dict
    if in_obj['id'] not in append_dict['team_id']:    
    
        # Team object
        append_dict['team_category'].append(in_obj['category'])
        append_dict['team_createdDate'].append(in_obj['createdDate'])
        append_dict['team_description'].append(in_obj['description'])
        append_dict['team_id'].append(in_obj['id'])
        append_dict['team_lenderCount'].append(in_obj['lenderCount'])
        append_dict['team_lentAmount'].append(in_obj['lentAmount'])
        append_dict['team_loanBecause'].append(in_obj['loanBecause'])
        append_dict['team_loanCount'].append(in_obj['loanCount'])
        append_dict['team_membershipType'].append(in_obj['membershipType'])
        append_dict['team_name'].append(in_obj['name'])
        append_dict['team_teamPublicId'].append(in_obj['teamPublicId'])
        append_dict['team_url'].append(in_obj['url'])
        append_dict['team_whereabouts'].append(in_obj['whereabouts'])

    return append_dict

#=============================================================================================================================
# Loan Actions Table
def action_obj_to_dict(in_obj, append_dict, loan_foreign_key):
    # Check if observation is not in dict
    if (in_obj['lender']['id'] not in append_dict['lender_id']) & (loan_foreign_key not in append_dict['loan_id']):   
        
        # LendingAction object
        append_dict['action_latestSharePurchaseDate'].append(in_obj['latestSharePurchaseDate'])
        append_dict['action_shareAmount'].append(in_obj['shareAmount'])
        append_dict['action_teams'].append(in_obj['teams'])

        # Foreign key - Lender
        append_dict['lender_id'].append(in_obj['lender']['id'])
        
        # Foreign key - Loan
        append_dict['loan_id'].append(loan_foreign_key)    
    
    return append_dict

#=============================================================================================================================
# Geocode Table
def geocode_obj_to_dict(in_obj, append_dict):
    # Check if observation is not in dict
    if (in_obj['latitude'] not in append_dict['geocode_latitude']) & (in_obj['longitude'] not in append_dict['geocode_longitude']):   
        
        # Geocode object    
        append_dict['geocode_city'].append(in_obj['city'])
        append_dict['geocode_latitude'].append(in_obj['latitude'])
        append_dict['geocode_longitude'].append(in_obj['longitude'])
        append_dict['geocode_postalCode'].append(in_obj['postalCode'])
        append_dict['geocode_state'].append(in_obj['state'])

        # Foreign key - Country
        append_dict['country_isoCode'].append(in_obj['country']['isoCode'])

    return append_dict

#=============================================================================================================================
# Countries Table
def country_obj_to_dict(in_obj, append_dict):
    # Check if observation is not in dict
    if in_obj['isoCode'] not in append_dict['country_isoCode']:   
    
        # Country object
        append_dict['country_fundsLentInCountry'].append(in_obj['fundsLentInCountry'])
        append_dict['country_isoCode'].append(in_obj['isoCode'])
        append_dict['country_name'].append(in_obj['name'])
        append_dict['country_numLoansFundraising'].append(in_obj['numLoansFundraising'])
        append_dict['country_ppp'].append(in_obj['ppp'])
        append_dict['country_region'].append(in_obj['region'])

    return append_dict

#=============================================================================================================================
# Sectors Table
def sector_obj_to_dict(in_obj, append_dict):
    # Check if observation is not in dict
    if in_obj['id'] not in append_dict['sector_id']:   
        append_dict['sector_id'].append(in_obj['id'])
        append_dict['sector_name'].append(in_obj['name'])
        
    return append_dict

#=============================================================================================================================
# Activities Table
def activity_obj_to_dict(in_obj, append_dict):
    # Check if observation is not in dict
    if in_obj['id'] not in append_dict['activity_id']:   
        append_dict['activity_id'].append(in_obj['id'])
        append_dict['activity_name'].append(in_obj['name'])
        
    return append_dict

### 4. Main function for parsing queries

In [ ]:
def split_query_to_dict(query,
                        loan_dict, 
                        term_dict, 
                        repayment_dict, 
                        borrower_dict, LoanBorrowerRelationship_dict, 
                        lender_dict, LoanLenderRelationship_dict, 
                        action_dict, 
                        team_dict, LenderTeamRelationship_dict, 
                        geocode_dict, country_dict, 
                        activity_dict, sector_dict):

    # Input Data
    data = query['data']['lend']['loans']['values']
    for data_element in data:
        n_lenders = data_element['lenders']['totalCount']
        n_lendingActions = data_element['lendingActions']['totalCount']
        #===============================================================================
        # 1. Loan object
        # 1.1 Append to dict
        loan_dict = loan_obj_to_dict(in_obj = data_element, 
                                     append_dict = loan_dict,
                                     n_lenders = n_lenders,
                                     n_lendingActions = n_lendingActions)

        #===============================================================================
        # 2. Loan Term object
        # 2.1 Extract values
        LoanTerm_obj = data_element['terms']
        # 2.2 Append to dict
        term_dict = term_obj_to_dict(in_obj = LoanTerm_obj, 
                                     append_dict = term_dict, 
                                     loan_foreign_key = data_element['id'])


        #===============================================================================
        # 3. Loan Repayment object 
        # 3.1 Extract values    
        RepaymentExpected_obj = LoanTerm_obj['expectedPayments'] # list of expected payments
        # 3.2 Append to dict
        for repayment_element in RepaymentExpected_obj:
            repayment_dict = repayment_obj_to_dict(in_obj = repayment_element, 
                                                   append_dict = repayment_dict, 
                                                   loan_foreign_key = data_element['id'])

        #===============================================================================
        # 4. Borrowers object
        # 4.1 Extract values 
        Borrower_obj = data_element['borrowers'] # list of borrowers
        # 4.2 Append to dict
        for borrower_element in Borrower_obj:        
            borrower_dict = borrower_obj_to_dict(in_obj = borrower_element, 
                                                   append_dict = borrower_dict)
            # 4.3 Create relationship dict       
            LoanBorrowerRelationship_dict['loan_id'].append(data_element['id'])
            LoanBorrowerRelationship_dict['borrower_id'].append(borrower_element['id'])  

        #===============================================================================
        # Check if loan was funded
        if n_lenders > 0:     

            #===============================================================================
            # 5. Lenders object
            # 5.1 Extract values 
            Lender_obj = data_element['lenders']['values'] # list of lenders
            #country = lenderPage['country']
            # 5.2 Append to dict
            for lender_element in Lender_obj:
                lender_dict = lender_obj_to_dict(in_obj = lender_element, 
                                                 append_dict = lender_dict)
                # 5.3 Create relationship dict 
                LoanLenderRelationship_dict['loan_id'].append(data_element['id'])
                LoanLenderRelationship_dict['lender_id'].append(lender_element['id'])

                #===============================================================================
                # Check if lender is a member of a team
                n_teams = lender_element['teams']['totalCount']
                if n_teams > 0:
                    #===============================================================================
                    # 6. Teams object  
                    # 6.1 Extract values 
                    Team_obj = lender_element['teams']['values'] # list of teams
                    # 6.2 Append to dict
                    for team_element in Team_obj:
                        team_dict = team_obj_to_dict(in_obj = team_element,
                                                     append_dict = team_dict)
                        # 6.3 Create relationship dict 
                        LenderTeamRelationship_dict['lender_id'].append(lender_element['id'])
                        LenderTeamRelationship_dict['team_id'].append(team_element['id'])

        #===============================================================================            
        # Check if lending acctions are applied
        if n_lendingActions > 0:
            #===============================================================================            
            # 8. Lending Action object
            # 8.1 Extract values 
            LendingAction_obj = data_element['lendingActions']['values'] # list of lendingActions
            # 8.2 Append to dict
            for action_element in LendingAction_obj:
                action_dict = action_obj_to_dict(in_obj = action_element,
                                                 append_dict = action_dict,
                                                 loan_foreign_key = data_element['id'])

        #===============================================================================
        # 9. Geocode object
        # 9.1 Extract values 
        Geocode_obj = data_element['geocode']
        # 9.2 Append to dict
        geocode_dict = geocode_obj_to_dict(in_obj = Geocode_obj,
                                           append_dict = geocode_dict)

        #===============================================================================
        # 10. Country object
        # 10.1 Extract values 
        Country_obj = data_element['geocode']['country']
        # 10.2 Append to dict
        country_dict = country_obj_to_dict(in_obj = Country_obj,
                                           append_dict = country_dict)

        #===============================================================================
        # 11. Activity object
        # 11.1 Extract values 
        Activity_obj = data_element['activity']
        # 11.2 Append to dict
        activity_dict = activity_obj_to_dict(in_obj = data_element['activity'],
                                             append_dict = activity_dict)
        
        #===============================================================================
        # 12. Sector object
        # 12.1 Extract values 
        Sector_obj = data_element['sector']
        # 12.2 Append to dict
        sector_dict = sector_obj_to_dict(in_obj = data_element['sector'], 
                                         append_dict = sector_dict)
        
    return loan_dict, term_dict, repayment_dict, borrower_dict, LoanBorrowerRelationship_dict, lender_dict, LoanLenderRelationship_dict, action_dict, team_dict, LenderTeamRelationship_dict, geocode_dict, country_dict, activity_dict, sector_dict

# II. Data Parsing

### 1. Query strycture

In [ ]:
# Query
base_url = '''
https://api.kivaws.org/graphql?query={
  lend {
    loans(
'''

# Object structure
loan_query = '''
    ) {
      values {
        anonymizationLevel
        borrowerCount
        dafEligible
        delinquent
        description
        descriptionInOriginalLanguage
        disbursalDate
        distributionModel
        fundraisingDate
        gender
        hasCurrencyExchangeLossLenders
        id
        inPfp
        isMatchable
        lenderRepaymentTerm
        loanAmount
        matcherAccountId
        matcherName
        matchingText
        matchRatio
        minNoteSize
        name
        paidAmount
        pfpMinLenders
        plannedExpirationDate
        previousLoanId
        raisedDate
        repaymentInterval
        researchScore
        status
        tags
        use
        whySpecial

        loanFundraisingInfo {
          fundedAmount
          isExpiringSoon
          reservedAmount
        }

        activity {
          id
          name
        }

        sector {
          id
          name
        }

        terms {
          currency
          currencyFullName
          disbursalAmount
          disbursalDate
          flexibleFundraisingEnabled
          lenderRepaymentTerm
          loanAmount
          lossLiabilityCurrencyExchange
          lossLiabilityNonpayment

          expectedPayments {
            amount
            dueToKivaDate
            effectiveDate
            localAmount
          }
        }

        borrowers {
          borrowedAmount
          firstName
          gender
          id
          isPrimary
          pictured
        }

        lenders(offset: 0, limit: 1000) {
          totalCount
          values {
            id
            inviteeCount
            loanCount
            memberSince
            name
            publicId

            lenderPage {
              city
              loanBecause
              occupation
              otherInfo
              state
              url
              whereabouts
              
              country {
                name
                isoCode
              }
            }

            teams(offset: 0, limit: 1000) {
              totalCount
              values {
                category
                createdDate
                description
                id
                lenderCount
                lentAmount
                loanBecause
                loanCount
                membershipType
                name
                teamPublicId
                url
                whereabouts
              }
            }
          }
        }

        lendingActions(offset: 0, limit: 1000) {
          totalCount
          values {
            latestSharePurchaseDate
            lender {
              id
            }
            shareAmount
            teams
          }
        }
        geocode {
          city
          state
          postalCode
          latitude
          longitude

          country {
            name
            isoCode
            region
            ppp
            numLoansFundraising
            fundsLentInCountry
          }
        }
      }
    }
  }
}

'''

### 2. Empty dictionaries for storing data

In [ ]:
# Create empty dictionaries 
loan_dict, term_dict, repayment_dict, borrower_dict, LoanBorrowerRelationship_dict, lender_dict, LoanLenderRelationship_dict, action_dict, team_dict, LenderTeamRelationship_dict, geocode_dict, country_dict, activity_dict, sector_dict = create_empty_dict()

### 3. Data Extraction

In [ ]:
number = 1  
while number < n_batches+1 :
    #===============================================================================
    # 1. Fetch data
    start = time.time()
    response = requests.get(base_url + filter_query + loan_query)
    query = response.json()
    #===============================================================================
    # 2. Split data into corresponding dictionaries  
    mid = time.time()
    loan_dict, term_dict, repayment_dict, borrower_dict, LoanBorrowerRelationship_dict, lender_dict, LoanLenderRelationship_dict, action_dict, team_dict, LenderTeamRelationship_dict, geocode_dict, country_dict, activity_dict, sector_dict = split_query_to_dict(query, loan_dict, term_dict, repayment_dict, borrower_dict, LoanBorrowerRelationship_dict, lender_dict, LoanLenderRelationship_dict, action_dict, team_dict, LenderTeamRelationship_dict, geocode_dict, country_dict, activity_dict, sector_dict)
    end = time.time()
    #===============================================================================
    # 3. Print info
    print('Batch: ', number, ' Query time:', round(mid - start, 2), ' Parsing time:', round(end - mid, 2) )
    #===============================================================================
    # 4. Increment step
    number = number+1

### 4. Transform into df

In [ ]:
loan_df = pd.DataFrame(loan_dict)
term_df = pd.DataFrame(term_dict)
repayment_df = pd.DataFrame(repayment_dict)
borrower_df = pd.DataFrame(borrower_dict)
LoanBorrowerRelationship_df = pd.DataFrame(LoanBorrowerRelationship_dict)
lender_df = pd.DataFrame(lender_dict)
LoanLenderRelationship_df = pd.DataFrame(LoanLenderRelationship_dict)
team_df = pd.DataFrame(team_dict)
LenderTeamRelationship_df = pd.DataFrame(LenderTeamRelationship_dict)
action_df = pd.DataFrame(action_dict)
geocode_df = pd.DataFrame(geocode_dict)
country_df = pd.DataFrame(country_dict)
activity_df = pd.DataFrame(activity_dict)
sector_df = pd.DataFrame(sector_dict)

# Correct lists
action_df['action_teams'] = action_df['action_teams'].astype(str).str.replace('[','',regex=False).str.replace(']','',regex=False)
loan_df['loan_tags'] = loan_df['loan_tags'].astype(str).str.replace('[','',regex=False).str.replace(']','',regex=False)

In [ ]:
# Short summary on table size
print(len(loan_df))
print(len(term_df))
print(len(repayment_df))
print(len(borrower_df))
print(len(LoanBorrowerRelationship_df))
print(len(lender_df))
print(len(LoanLenderRelationship_df))
print(len(team_df))
print(len(LenderTeamRelationship_df))
print(len(action_df))
print(len(geocode_df))
print(len(country_df))
print(len(activity_df))
print(len(sector_df))

### 6. Export Data

In [ ]:
loan_df.to_csv('Data/Loans.csv', sep=';')
term_df.to_csv('Data/Terms.csv', sep=';')
repayment_df.to_csv('Data/Repayments.csv', sep=';')
borrower_df.to_csv('Data/Borrowers.csv', sep=';')
LoanBorrowerRelationship_df.to_csv('Data/LoanBorrowerRelationship.csv', sep=';')
lender_df.to_csv('Data/Lenders.csv', sep=';')
LoanLenderRelationship_df.to_csv('Data/LoanLenderRelationship.csv', sep=';')
team_df.to_csv('Data/Teams.csv', sep=';')
LenderTeamRelationship_df.to_csv('Data/LenderTeamRelationship.csv', sep=';')
action_df.to_csv('Data/LoanActions.csv', sep=';')
geocode_df.to_csv('Data/Geocodes.csv', sep=';')
country_df.to_csv('Data/Countries.csv', sep=';')
activity_df.to_csv('Data/Activities.csv', sep=';')
sector_df.to_csv('Data/Sectors.csv', sep=';')